In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle

## Set parameters

In [2]:
# Set parameters
min_allowed_occurence = 10
max_allowed_frequency_rate = 0.95
labels = ['negative', 'positive']
destination = 'django_project/models/'

## Load and transform data

In [3]:
# Load data
df = pd.read_csv('data.csv')
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2, random_state=42)

# Transform features to bag of word representation
vec = TfidfVectorizer(min_df = min_allowed_occurence, max_df = max_allowed_frequency_rate)
vec.fit(X_train)
X_train_features = vec.transform(X_train)
X_test_features = vec.transform(X_test)

## Train a suport vector classifier

In [4]:
from sklearn.svm import LinearSVC

#Create to SCV
SVC = LinearSVC()
SVC.fit(X_train_features, y_train)
print(SVC.score(X_test_features, y_test))

0.77915625


## Save models 

In [5]:
# Save model using pickle
pickle.dump(SVC, open(destination+'model.sav', 'wb'))
pickle.dump(vec, open(destination+'vectorizer.pk', 'wb'))